# RESUME PROCESSING

In [1]:
pip install docx

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import Module
import os
import hashlib
import re
import nltk
from nltk.tokenize import RegexpTokenizer
import docx  # pip install python-docx
from nltk.corpus import stopwords
import pandas as pd
nltk.download('omw-1.4')
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/anudeepchowdary/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
# Folder path
path = "/Users/anudeepchowdary/Desktop/pre_resume/Resumes"

In [4]:
# Removing the duplicate files
def remove_duplicate():
    unique_files = []
    os.chdir("/Users/anudeepchowdary/Desktop/pre_resume/Resumes")
    for filename in os.listdir():
        if os.path.isfile(filename):
            filehash = hashlib.md5(open(
            filename,'rb').read()).hexdigest()
            if filehash not in unique_files:
                unique_files.append(filehash)
            else:
                os.remove(filename)
                print(f"{filename} has been deleted")

In [5]:
# Function to convert tokens to bigram words
def get_bigrams(sentence):
    tokens = nltk.word_tokenize(sentence)
    return zip(tokens, tokens[1:])


In [6]:
# Function to find the bigram tokens from text
def filter_bigram_tokens(bigram_list, bigram_tokens):
    token=[]
    for i in range(len(bigram_tokens)):
        if bigram_tokens[i] in bigram_list:
            token.append(bigram_tokens[i])
            i+=2
        else:
            word = bigram_tokens[i][:bigram_tokens[i].find(" ")]
            token.append(word)
    return token


In [7]:

def text_processing(bigram_list):
    # Change the directory
    os.chdir(path)
    filename = []
    resume_text = []
    tokens_list = []
    pre_token = []

    print("\n\n Resume Processing\n++++++++++++++++++++++++++++++++++++++++++\n")
    # iterate through all file
    for file in os.listdir():

        # Check whether file is in text format or not
        if file.endswith(".docx"):
            file_path = f"{path}\{file}"
            text = []
            try:
                doc = docx.Document(file_path)

                #Reading the document content
                for para in doc.paragraphs:
                    text.append(para.text)
                print(f"file: {file}")

                # Tokenizing the text
                tokenizer = RegexpTokenizer(r'\w+')
                tokens = tokenizer.tokenize(" ".join(text))
                tokens = [token for token in tokens if token not in stopwords.words('english')]

                # Removing the duplicate file with similar tokens and filtering bigram words
                token_text = " ".join(tokens)
                bigram_tokens = [' '.join(b) for b in get_bigrams(token_text.lower())]
                f_tokens = filter_bigram_tokens(bigram_list, bigram_tokens)
                lemmatizer = WordNetLemmatizer()
                lem_sent = [lemmatizer.lemmatize(words_sent) for words_sent in f_tokens]
                if pre_token[:10] != lem_sent[:10] and len(lem_sent)!=0:
                    # appending the text
                    print(lem_sent)
                    filename.append(file)
                    resume_text.append(text)
                    tokens_list.append(lem_sent)
                    pre_token=lem_sent
            except:
                continue

    # Change the directory
    os.chdir("/Users/anudeepchowdary/Desktop/pre_resume")
    # dictionary of lists
    resume_dict = {'File name': filename, 'Text': resume_text, 'Token_list': tokens_list}
    df = pd.DataFrame(resume_dict)
    # saving the dataframe
    df.to_csv('Resume_list.csv')

In [8]:
remove_duplicate()
os.chdir("/Users/anudeepchowdary/Desktop/pre_resume")

In [9]:
bigram_list= []
# retreiving the bigram words
f = open('bigram_words.txt', 'r')
word_list = f.readlines()
for sub in word_list:
    bigram_list.append(re.sub('\n', '', sub))

In [10]:
text_processing(bigram_list)



 Resume Processing
++++++++++++++++++++++++++++++++++++++++++

